# Section 2 Part 1


# User-User CF

**Zeyad Mohamed Mahmoud Fayed 222 102 242**

# Case Study 1

In [40]:
import pandas as 
import numpy as np

MIN_USER_RATINGS = 5
MIN_ITEM_RATINGS = 10

print("Original Dataset Shape:", ratings.shape)


df_clean = ratings.copy()
done = False
iteration = 0

while not done:
    iteration += 1
    start_shape = df_clean.shape
    

    user_counts = df_clean['user_id'].value_counts()
    active_users = user_counts[user_counts >= MIN_USER_RATINGS].index
    df_clean = df_clean[df_clean['user_id'].isin(active_users)]
    

    item_counts = df_clean['item_id'].value_counts()
    popular_items = item_counts[item_counts >= MIN_ITEM_RATINGS].index
    df_clean = df_clean[df_clean['item_id'].isin(popular_items)]
    
    end_shape = df_clean.shape
    
    print(f"Iteration {iteration}: {start_shape} -> {end_shape}")
    

    if start_shape == end_shape:
        done = True

print("\nFinal Filtered Dataset Shape:", df_clean.shape)

n_users = df_clean['user_id'].nunique()
n_items = df_clean['item_id'].nunique()
n_ratings = len(df_clean)
sparsity = (1 - (n_ratings / (n_users * n_items))) * 100

print(f"New Users: {n_users}")
print(f"New Items: {n_items}")
print(f"New Sparsity: {sparsity:.4f}% (Improved density)")

Original Dataset Shape: (1584082, 4)
Iteration 1: (1584082, 4) -> (190470, 4)
Iteration 2: (190470, 4) -> (115637, 4)
Iteration 3: (115637, 4) -> (102336, 4)
Iteration 4: (102336, 4) -> (98662, 4)
Iteration 5: (98662, 4) -> (97477, 4)
Iteration 6: (97477, 4) -> (97161, 4)
Iteration 7: (97161, 4) -> (97087, 4)
Iteration 8: (97087, 4) -> (97071, 4)
Iteration 9: (97071, 4) -> (97071, 4)

Final Filtered Dataset Shape: (97071, 4)
New Users: 9893
New Items: 4029
New Sparsity: 99.7565% (Improved density)


In [42]:
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random
import os

WORK_DIR = '/kaggle/working/User_CF_Results'
os.makedirs(WORK_DIR, exist_ok=True)

print("1. Preparing Optimized Matrix...")


unique_users = df_clean['user_id'].unique()
unique_items = df_clean['item_id'].unique()

user_to_idx = {user: i for i, user in enumerate(unique_users)}
idx_to_user = {i: user for user, i in user_to_idx.items()}

item_to_idx = {item: i for i, item in enumerate(unique_items)}
idx_to_item = {i: item for item, i in item_to_idx.items()}


user_indices = df_clean['user_id'].map(user_to_idx)
item_indices = df_clean['item_id'].map(item_to_idx)


sparse_matrix = csr_matrix(
    (df_clean['rating'], (user_indices, item_indices)),
    shape=(len(unique_users), len(unique_items))
)

print(f"Sparse Matrix Shape: {sparse_matrix.shape}")


print("2. Training SVD Model...")

svd = TruncatedSVD(n_components=50, random_state=42)
matrix_svd = svd.fit_transform(sparse_matrix)

print(f"SVD Reduced Matrix: {matrix_svd.shape} (Ready for Similarity)")


def get_recommendations(target_user_id, k_neighbors=50, top_n=10):
    # 1. الحصول على متجه المستخدم
    target_idx = user_to_idx[target_user_id]
    target_vector = matrix_svd[target_idx].reshape(1, -1)
    
    
    sim_scores = cosine_similarity(target_vector, matrix_svd).flatten()
    
    
    similar_indices = sim_scores.argsort()[::-1]
    similar_indices = similar_indices[similar_indices != target_idx]
    top_k_indices = similar_indices[:k_neighbors]
    
    
    print(f"\nTop 5 Neighbors for {target_user_id}:")
    for i in range(5):
        idx = top_k_indices[i]
        print(f"  User: {idx_to_user[idx]} | Similarity: {sim_scores[idx]:.4f}")

    
    neighbor_sims = sim_scores[top_k_indices]
    neighbor_ratings = sparse_matrix[top_k_indices] # (K, Items)
    
   
    weighted_sum = neighbor_ratings.T.dot(neighbor_sims)
    sum_of_sims = np.sum(np.abs(neighbor_sims))
    
    if sum_of_sims == 0: return None
    
    predicted_scores = weighted_sum / sum_of_sims
    
    
    user_history = sparse_matrix[target_idx].toarray().flatten()
    predicted_scores[user_history > 0] = -1 
    
    top_item_indices = predicted_scores.argsort()[::-1][:top_n]
    
    recs = []
    for idx in top_item_indices:
        if predicted_scores[idx] > 0:
            recs.append({
                'Item_ID': idx_to_item[idx],
                'Predicted_Rating': round(predicted_scores[idx], 4)
            })
            
    return pd.DataFrame(recs)


random_user = random.choice(unique_users)

print(f"\n--- Generating Recommendations for User: {random_user} ---")
recs_df = get_recommendations(random_user)

if recs_df is not None and not recs_df.empty:
    print("\nTop 10 Recommended Items:")
    display(recs_df)
    
   
    recs_df.to_csv(f'{WORK_DIR}/SVD_Recs_{random_user}.csv', index=False)
    print("Results saved.")
else:
    print("No recommendations found.")

1. Preparing Optimized Matrix...
Sparse Matrix Shape: (9893, 4029)
2. Training SVD Model...
SVD Reduced Matrix: (9893, 50) (Ready for Similarity)

--- Generating Recommendations for User: AYXA1QO0KIXWV ---

Top 5 Neighbors for AYXA1QO0KIXWV:
  User: A3DO1Z69FZKSIX | Similarity: 0.9911
  User: A3CPVP2M3JI6AN | Similarity: 0.9882
  User: A3AMABALFJBHDU | Similarity: 0.9858
  User: A2X46TS9VTP3BK | Similarity: 0.9855
  User: A24D0DSUY34824 | Similarity: 0.9809

Top 10 Recommended Items:


,Item_ID,Predicted_Rating
0,B00137VCQK,1.96
1,B00137MOS0,1.13
2,B00137Z0EK,0.77
3,B00MR8YM4I,0.68
4,B007QNRBRY,0.50
5,B004UE3784,0.50
6,B002X9RH2K,0.48
7,B000S51XMQ,0.41
8,B006H1ME5A,0.41
9,B00137UV44,0.40


Results saved.


In [43]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import os


WORK_DIR = '/kaggle/working/User_CF_Results'
os.makedirs(WORK_DIR, exist_ok=True)
print(f"Results will be saved to: {WORK_DIR}")



if 'df_clean' in globals():
    unique_users = df_clean['user_id'].unique()
    
    new_targets = df_clean['user_id'].value_counts().head(3).index.tolist()
    
    target_users_df = pd.DataFrame({
        'target': ['U1_New', 'U2_New', 'U3_New'],
        'user_id': new_targets
    })
    print(f"\nSelected New Target Users (from filtered data):\n{target_users_df}")
else:
    raise ValueError("df_clean not found! Please run the Iterative Filtering step first.")


print("\nCreating Matrices & Applying SVD...")


user_to_idx = {user: i for i, user in enumerate(df_clean['user_id'].unique())}
idx_to_user = {i: user for user, i in user_to_idx.items()}

item_to_idx = {item: i for i, item in enumerate(df_clean['item_id'].unique())}
idx_to_item = {i: item for item, i in item_to_idx.items()}


user_indices = df_clean['user_id'].map(user_to_idx)
item_indices = df_clean['item_id'].map(item_to_idx)


sparse_matrix = csr_matrix(
    (df_clean['rating'], (user_indices, item_indices)),
    shape=(len(user_to_idx), len(item_to_idx))
)


svd = TruncatedSVD(n_components=50, random_state=42)
matrix_svd = svd.fit_transform(sparse_matrix)

print(f"Sparse Matrix Shape: {sparse_matrix.shape}")
print(f"SVD Matrix Shape: {matrix_svd.shape}")


def run_user_based_cf(target_u_id, k_neighbors=50, n_recommendations=10):
    
    
    if target_u_id not in user_to_idx:
        print(f"User {target_u_id} not found in matrix.")
        return None, None

    target_idx = user_to_idx[target_u_id]
    
    
    target_vector_svd = matrix_svd[target_idx].reshape(1, -1)
    sim_scores = cosine_similarity(target_vector_svd, matrix_svd).flatten()
    
    
    similar_indices = sim_scores.argsort()[::-1]
    similar_indices = similar_indices[similar_indices != target_idx] # استبعاد النفس
    top_k_indices = similar_indices[:k_neighbors]
    
   
    neighbors_data = []
    for idx in top_k_indices:
        neighbors_data.append({
            'Target_User': target_u_id,
            'Neighbor_User_ID': idx_to_user[idx],
            'Similarity_Score': round(sim_scores[idx], 4)
        })
    neighbors_df = pd.DataFrame(neighbors_data)

    
    neighbor_sims = sim_scores[top_k_indices].reshape(-1, 1) 
    neighbor_ratings = sparse_matrix[top_k_indices]          
    
   
    
    weighted_sum = neighbor_ratings.T.dot(neighbor_sims)
    weighted_sum = np.asarray(weighted_sum).ravel() 
    
    
    sum_of_sims = np.sum(np.abs(neighbor_sims))
    
    if sum_of_sims == 0:
        predicted_ratings = np.zeros(sparse_matrix.shape[1])
    else:
        predicted_ratings = weighted_sum / sum_of_sims

   
    items_rated_by_target = sparse_matrix[target_idx].nonzero()[1]
    predicted_ratings[items_rated_by_target] = -1
    
   
    top_n_indices = predicted_ratings.argsort()[::-1][:n_recommendations]
    
    recs_data = []
    for idx in top_n_indices:
        score = predicted_ratings[idx]
        if score > 0:
            recs_data.append({
                'Target_User': target_u_id,
                'Item_ID': idx_to_item[idx],
                'Predicted_Rating': round(score, 4)
            })
            
    recs_df = pd.DataFrame(recs_data)
    
    return neighbors_df, recs_df


print("\nProcessing Target Users...")

for i, row in target_users_df.iterrows():
    label = row['target']
    user_id = row['user_id']
    
    print(f"\nRunning User CF (SVD-Enhanced) for {label}: {user_id}")
    
    neighbors_df, recs_df = run_user_based_cf(user_id, k_neighbors=50)
    
    
    if neighbors_df is not None and not neighbors_df.empty:
        n_filename = f"{label}_Neighbors.csv"
        neighbors_df.to_csv(os.path.join(WORK_DIR, n_filename), index=False)
        print(f"  -> Saved neighbors to {n_filename}")
        display(neighbors_df.head(3))
        
    
    if recs_df is not None and not recs_df.empty:
        r_filename = f"{label}_Recommendations.csv"
        recs_df.to_csv(os.path.join(WORK_DIR, r_filename), index=False)
        print(f"  -> Saved recommendations to {r_filename}")
        display(recs_df.head(3))
    else:
        print("  -> No recommendations found.")

print("\n" + "="*50)
print(f"All files saved in: {WORK_DIR}")
print("="*50

Results will be saved to: /kaggle/working/User_CF_Results

Selected New Target Users (from filtered data):
   target         user_id
0  U1_New  A3W4D8XOGLWUN5
1  U2_New  A2H3JURQZOHVMB
2  U3_New  A36EDWL4F3AASU

Creating Matrices & Applying SVD...
Sparse Matrix Shape: (9893, 4029)
SVD Matrix Shape: (9893, 50)

Processing Target Users...

Running User CF (SVD-Enhanced) for U1_New: A3W4D8XOGLWUN5
  -> Saved neighbors to U1_New_Neighbors.csv


,Target_User,Neighbor_User_ID,Similarity_Score
0,A3W4D8XOGLWUN5,AK0H0GRSKKOBO,0.89
1,A3W4D8XOGLWUN5,A3IY9HIAMJQ7HL,0.87
2,A3W4D8XOGLWUN5,A1IPYBGXRLS6MX,0.87


  -> Saved recommendations to U1_New_Recommendations.csv


,Target_User,Item_ID,Predicted_Rating
0,A3W4D8XOGLWUN5,B000W08K0U,0.35
1,A3W4D8XOGLWUN5,B001229BCS,0.23
2,A3W4D8XOGLWUN5,B000WLH8OI,0.22



Running User CF (SVD-Enhanced) for U2_New: A2H3JURQZOHVMB
  -> Saved neighbors to U2_New_Neighbors.csv


...


Running User CF (SVD-Enhanced) for U3_New: A36EDWL4F3AASU
  -> Saved neighbors to U3_New_Neighbors.csv


,Target_User,Neighbor_User_ID,Similarity_Score
0,A36EDWL4F3AASU,A1KYHC3ZXAJICU,0.82
1,A36EDWL4F3AASU,AXG5I4CK8OQ2F,0.70
2,A36EDWL4F3AASU,ATIDIWJFL41W7,0.69


  -> Saved recommendations to U3_New_Recommendations.csv


,Target_User,Item_ID,Predicted_Rating
0,A36EDWL4F3AASU,B00961ZGQC,0.76
1,A36EDWL4F3AASU,B00LWC6P1S,0.51
2,A36EDWL4F3AASU,B00A3SGY4K,0.51



All files saved in: /kaggle/working/User_CF_Results


In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import os
import math

WORK_DIR = '/kaggle/working/User_CF_Results'
os.makedirs(WORK_DIR, exist_ok=True)

TARGET_USER_ID = 'AAE2DUEMTR30I'

print("1. Performing Iterative Filtering (To fix Sparsity)...")


MIN_USER_RATINGS = 5
MIN_ITEM_RATINGS = 10

df_clean = ratings.copy()
done = False
iteration = 0

while not done:
    iteration += 1
    start_shape = df_clean.shape
    
    user_counts = df_clean['user_id'].value_counts()
    active_users = user_counts[user_counts >= MIN_USER_RATINGS].index
    df_clean = df_clean[df_clean['user_id'].isin(active_users)]
    
    item_counts = df_clean['item_id'].value_counts()
    popular_items = item_counts[item_counts >= MIN_ITEM_RATINGS].index
    df_clean = df_clean[df_clean['item_id'].isin(popular_items)]
    
    end_shape = df_clean.shape
    print(f"   Iteration {iteration}: {start_shape} -> {end_shape}")
    
    if start_shape == end_shape:
        done = True

print(f"\nFinal Dataset Shape: {df_clean.shape}")

if TARGET_USER_ID not in df_clean['user_id'].values:
    print(f"Warning: Target user {TARGET_USER_ID} was filtered out! Selecting a new top user...")
    TARGET_USER_ID = df_clean['user_id'].value_counts().index[0]
    print(f"New Target User: {TARGET_USER_ID}")


print("\n2. Creating Matrix & Applying SVD...")

user_ids = df_clean['user_id'].unique()
item_ids = df_clean['item_id'].unique()

user_to_idx = {user: i for i, user in enumerate(user_ids)}
item_to_idx = {item: i for i, item in enumerate(item_ids)}

user_indices = df_clean['user_id'].map(user_to_idx)
item_indices = df_clean['item_id'].map(item_to_idx)

sparse_matrix = csr_matrix(
    (df_clean['rating'], (user_indices, item_indices)),
    shape=(len(user_ids), len(item_ids))
)

svd = TruncatedSVD(n_components=50, random_state=42)
matrix_svd = svd.fit_transform(sparse_matrix)

print(f"Sparse Shape: {sparse_matrix.shape}")
print(f"SVD Shape: {matrix_svd.shape}")

print(f"\n3. Calculating SVD-Based Similarity for {TARGET_USER_ID}...")

target_idx = user_to_idx[TARGET_USER_ID]
target_vector = matrix_svd[target_idx].reshape(1, -1)

similarity_scores = cosine_similarity(target_vector, matrix_svd).flatten()

results_df = pd.DataFrame({
    'User_ID': user_ids,
    'Similarity_Score': similarity_scores
})

results_df = results_df[results_df['User_ID'] != TARGET_USER_ID]

results_df = results_df[results_df['Similarity_Score'] > 0]

results_df = results_df.sort_values(by='Similarity_Score', ascending=False)

top_k_count = math.ceil(len(results_df) * 0.20)
top_20_df = results_df.head(top_k_count)

output_file = os.path.join(WORK_DIR, f'Similarity_SVD_{TARGET_USER_ID}.csv')
top_20_df.to_csv(output_file, index=False)

print(f"Done! Top {len(top_20_df)} users saved to: {output_file}")

print("\nTop 10 Similar Users (SVD Results):")
display(top_20_df.head(10))

1. Performing Iterative Filtering (To fix Sparsity)...
   Iteration 1: (1584082, 4) -> (190470, 4)
   Iteration 2: (190470, 4) -> (115637, 4)
   Iteration 3: (115637, 4) -> (102336, 4)
   Iteration 4: (102336, 4) -> (98662, 4)
   Iteration 5: (98662, 4) -> (97477, 4)
   Iteration 6: (97477, 4) -> (97161, 4)
   Iteration 7: (97161, 4) -> (97087, 4)
   Iteration 8: (97087, 4) -> (97071, 4)
   Iteration 9: (97071, 4) -> (97071, 4)

Final Dataset Shape: (97071, 4)
New Target User: A3W4D8XOGLWUN5

2. Creating Matrix & Applying SVD...
Sparse Shape: (9893, 4029)
SVD Shape: (9893, 50)

3. Calculating SVD-Based Similarity for A3W4D8XOGLWUN5...
Done! Top 1420 users saved to: /kaggle/working/User_CF_Results/Similarity_SVD_A3W4D8XOGLWUN5.csv

Top 10 Similar Users (SVD Results):


,User_ID,Similarity_Score
7618,AK0H0GRSKKOBO,0.89
6177,A3IY9HIAMJQ7HL,0.87
6176,A1IPYBGXRLS6MX,0.87
1954,A3QOQY24XUTR28,0.87
3778,AD5R0C4EO8M9E,0.84
7812,A36XJ5PAWMXHCV,0.82
5786,A1NQFV0KQPM366,0.82
3563,A1Z0KQUE6A5F9,0.81
2031,AIYB8FUXTF365,0.81
530,A368R6C5ZEO71R,0.81


In [46]:
import pandas as pd
import math
import os
from sklearn.metrics.pairwise import cosine_similarity


WORK_DIR = '/kaggle/working/User_CF_Results'


if 'user_to_idx' not in globals():
    raise ValueError("User mappings not found. Please run the SVD/Matrix step first.")


REQUESTED_USER_ID = 'AAE2DUEMTR30I'

if REQUESTED_USER_ID in user_to_idx:
    TARGET_USER_ID = REQUESTED_USER_ID
    print(f"User {TARGET_USER_ID} found in filtered data.")
else:
    print(f"⚠️ User {REQUESTED_USER_ID} was dropped during data filtering.")
    

    if 'df_clean' in globals():
        TARGET_USER_ID = df_clean['user_id'].value_counts().idxmax()
    else:
        TARGET_USER_ID = list(user_to_idx.keys())[0]
        
    print(f"-> Switched to NEW Valid Target User: {TARGET_USER_ID}")

print(f"\nIdentifying Top 20% neighbors (SVD-Based) for: {TARGET_USER_ID}")


target_idx = user_to_idx[TARGET_USER_ID]

#
target_vector_svd = matrix_svd[target_idx].reshape(1, -1)


sim_scores = cosine_similarity(target_vector_svd, matrix_svd).flatten()


valid_user_ids = list(user_to_idx.keys())

df_scores = pd.DataFrame({
    'User_ID': valid_user_ids,
    'Similarity_Score': sim_scores
})


df_scores = df_scores[df_scores['User_ID'] != TARGET_USER_ID]


df_scores = df_scores[df_scores['Similarity_Score'] > 0]

df_scores = df_scores.sort_values(by='Similarity_Score', ascending=False)

total_neighbors = len(df_scores)
top_20_count = math.ceil(total_neighbors * 0.20)

print(f"Total valid neighbors (Sim > 0): {total_neighbors}")
print(f"Top 20% count: {top_20_count}")

# هـ) استخراج القائمة
top_20_percent_df = df_scores.head(top_20_count)

output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_SVD_Top20_Neighbors.csv')
top_20_percent_df.to_csv(output_file, index=False)

print(f"\nSaved SVD Top 20% to: {output_file}")
print("\nHere are the Top 20% Users (SVD):")
display(top_20_percent_df.head(10

⚠️ User AAE2DUEMTR30I was dropped during data filtering.
-> Switched to NEW Valid Target User: A3W4D8XOGLWUN5

Identifying Top 20% neighbors (SVD-Based) for: A3W4D8XOGLWUN5
Total valid neighbors (Sim > 0): 7098
Top 20% count: 1420

Saved SVD Top 20% to: /kaggle/working/User_CF_Results/A3W4D8XOGLWUN5_SVD_Top20_Neighbors.csv

Here are the Top 20% Users (SVD):


,User_ID,Similarity_Score
7618,AK0H0GRSKKOBO,0.89
6177,A3IY9HIAMJQ7HL,0.87
6176,A1IPYBGXRLS6MX,0.87
1954,A3QOQY24XUTR28,0.87
3778,AD5R0C4EO8M9E,0.84
7812,A36XJ5PAWMXHCV,0.82
5786,A1NQFV0KQPM366,0.82
3563,A1Z0KQUE6A5F9,0.81
2031,AIYB8FUXTF365,0.81
530,A368R6C5ZEO71R,0.81


In [ ]:
import numpy as np
import pandas as pd
import os

# التأكد من المتغيرات
if 'top_20_percent_df' not in globals() or top_20_percent_df.empty:
    raise ValueError("Error: Top 20% neighbors dataframe missing. Run the previous step first.")

if 'sparse_matrix' not in globals():
    raise ValueError("Error: Sparse Matrix missing. Run the Matrix Creation step first.")


if 'TARGET_USER_ID' not in globals():
    print("Warning: TARGET_USER_ID variable lost. Using latest valid user.")
    TARGET_USER_ID = list(user_to_idx.keys())[0] 

print(f"Predicting ratings for: {TARGET_USER_ID}")
print(f"Based on {len(top_20_percent_df)} neighbors (SVD-selected)...")
neighbor_ids = top_20_percent_df['User_ID'].values
neighbor_sims = top_20_percent_df['Similarity_Score'].values

neighbor_indices = [user_to_idx[uid] for uid in neighbor_ids]

neighbor_ratings_matrix = sparse_matrix[neighbor_indices]

numerator = neighbor_ratings_matrix.T.dot(neighbor_sims)

binary_ratings = neighbor_ratings_matrix.copy()
binary_ratings.data = np.ones_like(binary_ratings.data) # 1 if rated, 0 if not
denominator = binary_ratings.T.dot(neighbor_sims)

predicted_ratings = np.zeros(sparse_matrix.shape[1])
mask = denominator > 0
predicted_ratings[mask] = numerator[mask] / denominator[mask]


target_idx = user_to_idx[TARGET_USER_ID]
items_rated_by_target = sparse_matrix[target_idx].indices

predicted_ratings[items_rated_by_target] = 0

recommended_item_indices = predicted_ratings.argsort()[::-1]

recommendations_list = []
count = 0

for idx in recommended_item_indices:
    score = predicted_ratings[idx]
    
    if score <= 0: break
    if count >= 50: brea
    
    recommendations_list.append({
        'Target_User': TARGET_USER_ID,
        'Item_ID': idx_to_item[idx], 
        'Predicted_Rating': round(score, 4)
    })
    count += 1

recs_df = pd.DataFrame(recommendations_list)

output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_SVD_Predicted_Preferences.csv')
recs_df.to_csv(output_file, index=False)

print(f"\nPrediction Complete!")
print(f"Number of unrated items predicted: {len(recs_df)}")
print(f"Results saved to: {output_file}")

if not recs_df.empty:
    print("\nTop 10 Recommended Items for User (SVD-Based):")
    display(recs_df.head(10))
el
    print("No recommendations generated.")

Predicting ratings for: A3W4D8XOGLWUN5
Based on 1420 neighbors (SVD-selected)...

Prediction Complete!
Number of unrated items predicted: 50
Results saved to: /kaggle/working/User_CF_Results/A3W4D8XOGLWUN5_SVD_Predicted_Preferences.csv

Top 10 Recommended Items for User (SVD-Based):


,Target_User,Item_ID,Predicted_Rating
0,A3W4D8XOGLWUN5,B00I83XD14,20.00
1,A3W4D8XOGLWUN5,B00123LX7I,15.60
2,A3W4D8XOGLWUN5,B00136NKHU,13.53
3,A3W4D8XOGLWUN5,B001416OXG,12.26
4,A3W4D8XOGLWUN5,B000W1MCQW,10.00
5,A3W4D8XOGLWUN5,B00123M8ZY,10.00
6,A3W4D8XOGLWUN5,B0013873K8,10.00
7,A3W4D8XOGLWUN5,B00123BCKQ,10.00
8,A3W4D8XOGLWUN5,B00137VDBY,10.00
9,A3W4D8XOGLWUN5,B00137VE9A,10.00


In [48]:
import numpy as np
import pandas as pd
import math
import os
from sklearn.metrics.pairwise import cosine_similarity

WORK_DIR = '/kaggle/working/User_CF_Result
if 'sparse_matrix' not in globals() or 'matrix_svd' not in globals():
    raise ValueError("Error: Filtered Matrices (sparse_matrix, matrix_svd) not found. Run the SVD step first.")

PREFERRED_ID = 'AAE2DUEMTR30I'

if PREFERRED_ID in user_to_idx:
    TARGET_USER_ID = PREFERRED_ID
else:
    TARGET_USER_ID = list(user_to_idx.keys())[0]
    print(f"Warning: {PREFERRED_ID} not in filtered data. Switched to {TARGET_USER_ID}")

print(f"Calculating Discounted Similarity (DS) for: {TARGET_USER_ID}"
target_idx = user_to_idx[TARGET_USER_ID]

target_vector_svd = matrix_svd[target_idx].reshape(1, -1)

svd_sim_scores = cosine_similarity(target_vector_svd, matrix_svd).flatten()
target_vector_sparse = sparse_matrix[target_idx]
num_target_ratings = target_vector_sparse.getnnz()

beta = math.ceil(0.30 * num_target_ratings)
beta = max(1, beta)


print(f"Target User Rated Items (Actual): {num_target_ratings}")
print(f"Threshold (Beta) @ 30%: {beta}")

print("Calculating actual overlap count...")
binary_matrix = sparse_matrix.copy()
binary_matrix.data[:] = 1
binary_target = binary_matrix[target_idx]

common_counts = binary_target.dot(binary_matrix.T).toarray().flatten()

user_ids_list = list(user_to_idx.keys())

ds_df = pd.DataFrame({
    'User_ID': user_ids_list,
    'Similarity_Score': svd_sim_scores, 
    
    'Num_Common_Items': common_counts  
    
})

ds_df['Discount_Factor'] = ds_df['Num_Common_Items'].apply(lambda x: min(x / beta, 1.0))

ds_df['Discounted_Similarity'] = ds_df['Similarity_Score'] * ds_df['Discount_Factor']


ds_df = ds_df[ds_df['User_ID'] != TARGET_USER_ID]

ds_valid = ds_df[ds_df['Discounted_Similarity'] > 0].sort_values(
    by='Discounted_Similarity', ascending=False
)

# حساب الـ 20%
total_valid = len(ds_valid)
top_k_count = math.ceil(total_valid * 0.20)

print(f"Total valid neighbors (DS > 0): {total_valid}")
print(f"Top 20% count: {top_k_count}")


top_20_svd_ds = ds_valid.head(top_k_count)

output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_SVD_DS_Top20.csv')
top_20_svd_ds.to_csv(output_file, index=False)

print(f"\nSaved SVD-DS Top 20% list to: {output_file}")

print("\nTop 10 Similar Users (SVD + Discounting):")
# عرض الأعمدة المهمة
cols = ['User_ID', 'Num_Common_Items', 'Discount_Factor', 'Similarity_Score', 'Discounted_Similarity']
display(top_20_svd_ds[cols].head

Calculating Discounted Similarity (DS) for: A9Q28YTLYREO7
Target User Rated Items (Actual): 70
Threshold (Beta) @ 30%: 21
Calculating actual overlap count...
Total valid neighbors (DS > 0): 977
Top 20% count: 196

Saved SVD-DS Top 20% list to: /kaggle/working/User_CF_Results/A9Q28YTLYREO7_SVD_DS_Top20.csv

Top 10 Similar Users (SVD + Discounting):


,User_ID,Num_Common_Items,Discount_Factor,Similarity_Score,Discounted_Similarity
44,A1QEWOSV05RYEO,17.00,0.81,0.75,0.60
2183,A3HU0B9XUEVHIM,14.00,0.67,0.83,0.56
7159,A1HCCW38EQQBTY,12.00,0.57,0.81,0.46
4206,A24N1BAS3CU27H,12.00,0.57,0.80,0.45
3385,AE31M52VLKOG6,11.00,0.52,0.83,0.43
7157,A1P41TBZBOLTY1,10.00,0.48,0.70,0.33
14,AD3C29MKDX7V2,8.00,0.38,0.87,0.33
9179,AJNQTSR3IMW8Z,8.00,0.38,0.82,0.31
3254,A3PCTD8QM1BIXI,7.00,0.33,0.87,0.29
1219,AMP7TQRWAIE84,11.00,0.52,0.53,0.28


In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

print("1. Preparing Data from Filtered Dataset...
if 'df_clean' not in globals():
    raise ValueError("Error: 'df_clean' not found. Please run the Iterative Filtering step first."
unique_users = df_clean['user_id'].unique()
unique_items = df_clean['item_id'].unique()

user_to_idx = {user: i for i, user in enumerate(unique_users)}
item_to_idx = {item: i for i, item in enumerate(unique_items)}
user_indices = df_clean['user_id'].map(user_to_idx)
item_indices = df_clean['item_id'].map(item_to_idx)

sparse_matrix = csr_matrix(
    (df_clean['rating'], (user_indices, item_indices)),
    shape=(len(unique_users), len(unique_items))
)

print(f"Sparse Matrix Shape: {sparse_matrix.shape}")

print("2. Applying SVD for Dimensionality Reduction...")

svd = TruncatedSVD(n_components=50, random_state=42)

matrix_svd = svd.fit_transform(sparse_matrix)

print(f"Reduced Matrix Shape (SVD): {matrix_svd.shape}") 


PREFERRED_ID = 'AAE2DUEMTR30I'

if PREFERRED_ID in user_to_idx:
    TARGET_USER_ID = PREFERRED_ID
    print(f"Target User Found: {TARGET_USER_ID}")
else:
    
    TARGET_USER_ID = df_clean['user_id'].value_counts().idxmax()
    print(f"Warning: {PREFERRED_ID} filtered out. Switched to Top User: {TARGET_USER_ID}")


print(f"3. Calculating Similarity for {TARGET_USER_ID}...")

target_idx = user_to_idx[TARGET_USER_ID]
target_vector_reduced = matrix_svd[target_idx].reshape(1, -1)


sim_scores_svd = cosine_similarity(target_vector_reduced, matrix_svd).flatten()


results_svd = pd.DataFrame({
    'User_ID': unique_users,
    'Similarity_Score': sim_scores_svd
})


results_svd = results_svd[results_svd['User_ID'] != TARGET_USER_ID]


results_svd = results_svd[results_svd['Similarity_Score'] > 0]


results_svd = results_svd.sort_values(by='Similarity_Score', ascending=False)

print(f"\nTop 10 Similar Users using SVD (Total found: {len(results_svd)}):")
display(results_svd.head

1. Preparing Data from Filtered Dataset...
Sparse Matrix Shape: (9893, 4029)
2. Applying SVD for Dimensionality Reduction...
Reduced Matrix Shape (SVD): (9893, 50)
3. Calculating Similarity for A3W4D8XOGLWUN5...

Top 10 Similar Users using SVD (Total found: 7098):


,User_ID,Similarity_Score
7618,AK0H0GRSKKOBO,0.89
6177,A3IY9HIAMJQ7HL,0.87
6176,A1IPYBGXRLS6MX,0.87
1954,A3QOQY24XUTR28,0.87
3778,AD5R0C4EO8M9E,0.84
7812,A36XJ5PAWMXHCV,0.82
5786,A1NQFV0KQPM366,0.82
3563,A1Z0KQUE6A5F9,0.81
2031,AIYB8FUXTF365,0.81
530,A368R6C5ZEO71R,0.81


In [ ]:
import numpy as np
import pandas as pd
import math
import os
from sklearn.metrics.pairwise import cosine_similarity


WORK_DIR = '/kaggle/working/User_CF_Results'
TARGET_USER_ID = 'A3W4D8XOGLWUN5' 

print(f"🚀 Fixing missing files and running comparison for: {TARGET_USER_ID}")


if 'sparse_matrix' not in globals() or 'matrix_svd' not in globals():
    raise ValueError("Error: Matrices missing. Please run the 'Iterative Filtering & SVD' step first.")

if TARGET_USER_ID not in user_to_idx:
    
    TARGET_USER_ID = list(user_to_idx.keys())[0]
    print(f"Warning: Specific user not found. Switched to: {TARGET_USER_ID}")


print("\n[1/3] Generating Discounted Similarity (DS) Neighbors...")

target_idx = user_to_idx[TARGET_USER_ID]

target_vector_svd = matrix_svd[target_idx].reshape(1, -1)
svd_sim_scores = cosine_similarity(target_vector_svd, matrix_svd).flatten()

binary_matrix = sparse_matrix.copy()
binary_matrix.data[:] = 1
common_counts = binary_matrix[target_idx].dot(binary_matrix.T).toarray().flatten()

num_target_ratings = sparse_matrix[target_idx].getnnz()
beta = max(1, math.ceil(0.30 * num_target_ratings))

user_ids_list = list(user_to_idx.keys())
ds_df = pd.DataFrame({
    'User_ID': user_ids_list,
    'Similarity_Score': svd_sim_scores,
    'Num_Common_Items': common_counts
})

ds_df['Discount_Factor'] = ds_df['Num_Common_Items'].apply(lambda x: min(x / beta, 1.0))
ds_df['Discounted_Similarity'] = ds_df['Similarity_Score'] * ds_df['Discount_Factor']

ds_df = ds_df[ds_df['User_ID'] != TARGET_USER_ID] 
ds_df = ds_df[ds_df['Discounted_Similarity'] > 0].sort_values(by='Discounted_Similarity', ascending=False)

top_k = math.ceil(len(ds_df) * 0.20)
ds_neighbors_df = ds_df.head(top_k)

ds_file_path = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_SVD_DS_Top20.csv')
ds_neighbors_df.to_csv(ds_file_path, index=False)
print(f"   -> Saved: {ds_file_path} ({len(ds_neighbors_df)} neighbors)")



print("\n[2/3] Generating Standard SVD Predictions...")

raw_neighbors = ds_df.sort_values(by='Similarity_Score', ascending=False).head(50)
neighbor_indices = [user_to_idx[uid] for uid in raw_neighbors['User_ID']]
neighbor_sims = raw_neighbors['Similarity_Score'].values.reshape(-1, 1)
neighbor_ratings = sparse_matrix[neighbor_indices]

weighted_sum = neighbor_ratings.T.dot(neighbor_sims).ravel()
sum_sims = np.sum(np.abs(neighbor_sims))
std_preds = weighted_sum / sum_sims if sum_sims > 0 else np.zeros(sparse_matrix.shape[1])

std_preds[sparse_matrix[target_idx].indices] = 0

top_idx = std_preds.argsort()[::-1][:50]
std_recs = [{'Item_ID': idx_to_item[i], 'Standard_SVD_Rating': round(std_preds[i], 4)} for i in top_idx if std_preds[i] > 0]
std_recs_df = pd.DataFrame(std_recs)

std_file_path = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_SVD_Predicted_Preferences.csv')
std_recs_df.to_csv(std_file_path, index=False)
print(f"   -> Saved: {std_file_path}")



print("\n[3/3] Comparing Predictions (Standard vs DS)...")


ds_indices = [user_to_idx[uid] for uid in ds_neighbors_df['User_ID']]
ds_weights = ds_neighbors_df['Discounted_Similarity'].values.reshape(-1, 1)
ds_ratings_mat = sparse_matrix[ds_indices]


ds_num = ds_ratings_mat.T.dot(ds_weights).ravel()
ds_denom = ds_ratings_mat.copy()
ds_denom.data[:] = 1
ds_div = ds_denom.T.dot(ds_weights).ravel()

ds_final_preds = np.zeros(sparse_matrix.shape[1])
mask = ds_div > 0
ds_final_preds[mask] = ds_num[mask] / ds_div[mask]
ds_final_preds[sparse_matrix[target_idx].indices] = 0

ds_top_idx = ds_final_preds.argsort()[::-1][:50]
ds_recs = [{'Item_ID': idx_to_item[i], 'DS_Rating': round(ds_final_preds[i], 4)} for i in ds_top_idx if ds_final_preds[i] > 0]
ds_recs_df = pd.DataFrame(ds_recs)

if not std_recs_df.empty and not ds_recs_df.empty:
    comparison_df = pd.merge(std_recs_df, ds_recs_df, on='Item_ID', how='inner')
    comparison_df['Diff'] = comparison_df['Standard_SVD_Rating'] - comparison_df['DS_Rating']
    
    print(f"\n=== Final Comparison ({len(comparison_df)} overlapping items) ===")
    if comparison_df.empty:
        print("No overlap! The recommendations are completely different.")
        print("Top Standard:", std_recs_df['Item_ID'].head(3).tolist())
        print("Top DS:", ds_recs_df['Item_ID'].head(3).tolist())
    else:
        display(comparison_df.head(10))
        
    comp_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_Prediction_Comparison.csv')
    comparison_df.to_csv(comp_file, index=False)
    print(f"\nComparison saved to: {comp_file}")
else:
    print("No recommendations generated to compare.")

🚀 Fixing missing files and running comparison for: A3W4D8XOGLWUN5

[1/3] Generating Discounted Similarity (DS) Neighbors...
   -> Saved: /kaggle/working/User_CF_Results/A3W4D8XOGLWUN5_SVD_DS_Top20.csv (955 neighbors)

[2/3] Generating Standard SVD Predictions...
   -> Saved: /kaggle/working/User_CF_Results/A3W4D8XOGLWUN5_SVD_Predicted_Preferences.csv

[3/3] Comparing Predictions (Standard vs DS)...

=== Final Comparison (3 overlapping items) ===


,Item_ID,Standard_SVD_Rating,DS_Rating,Diff
0,B000VZJPJ6,0.21,10.00,-9.79
1,B00122WK08,0.19,10.00,-9.81
2,B00122D76O,0.19,10.00,-9.81



Comparison saved to: /kaggle/working/User_CF_Results/A3W4D8XOGLWUN5_Prediction_Comparison.csv


In [ ]:
import pandas as pd
import numpy as np


WORK_DIR = '/kaggle/working/User_CF_Results'


print(f"Analyzing Neighbor Types for: {TARGET_USER_ID}")

target_items = set(df_clean[df_clean['user_id'] == TARGET_USER_ID]['item_id'])
print(f"Target User Rated: {len(target_items)} items")


if 'top_20_svd_ds' in globals():
    neighbors_df = top_20_svd_ds.copy()
else:
    
    file_path = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_SVD_DS_Top20.csv')
    if os.path.exists(file_path):
        neighbors_df = pd.read_csv(file_path)
    else:
        raise ValueError("Neighbors data not found. Run previous steps first.")


neighbor_ids = neighbors_df['User_ID'].tolist()
neighbor_activity = df_clean[df_clean['user_id'].isin(neighbor_ids)].groupby('user_id')['item_id'].count()

neighbors_df['Total_Neighbor_Ratings'] = neighbors_df['User_ID'].map(neighbor_activity)
neighbors_df['Num_Common_Items'] = neighbors_df['Num_Common_Items'].astype(int) # Ensure int

neighbors_df['Overlap_Ratio'] = round(neighbors_df['Num_Common_Items'] / neighbors_df['Total_Neighbor_Ratings'], 2)

def classify_trust(row):
    if row['Overlap_Ratio'] >= 0.8:
        return "Twin (High Overlap %)"
    elif row['Overlap_Ratio'] <= 0.2:
        return "Explorer (Low Overlap %, Many other items)"
    else:
        return "Balanced"

neighbors_df['User_Type'] = neighbors_df.apply(classify_trust, axis=1)


print("\n--- Group A: The 'Twins' (Rated mostly common items) ---")
display(neighbors_df[neighbors_df['User_Type'].str.contains("Twin")].head(5))

print("\n--- Group B: The 'Explorers' (Rated common items + MANY others) ---")
display(neighbors_df[neighbors_df['User_Type'].str.contains("Explorer")].head(5))

# Save for analysis
neighbors_df.to_csv(os.path.join(WORK_DIR, f'{TARGET_USER_ID}_Trust_Analysis.csv'), index=False)

Analyzing Neighbor Types for: AAE2DUEMTR30I
Target User Rated: 0 items

--- Group A: The 'Twins' (Rated mostly common items) ---


,User_ID,Similarity_Score,Num_Common_Items,Discount_Factor,Discounted_Similarity,Total_Neighbor_Ratings,Overlap_Ratio,User_Type
14,AD3C29MKDX7V2,0.87,8,0.38,0.33,9,0.89,Twin (High Overlap %)
9179,AJNQTSR3IMW8Z,0.82,8,0.38,0.31,8,1.00,Twin (High Overlap %)
5194,A23Y7ESOGEBSZT,0.87,4,0.19,0.17,5,0.80,Twin (High Overlap %)
8008,A311XNVMNZHKM8,0.85,4,0.19,0.16,5,0.80,Twin (High Overlap %)
8321,A66WBGGY7AYQ5,0.85,4,0.19,0.16,5,0.80,Twin (High Overlap %)



--- Group B: The 'Explorers' (Rated common items + MANY others) ---


,User_ID,Similarity_Score,Num_Common_Items,Discount_Factor,Discounted_Similarity,Total_Neighbor_Ratings,Overlap_Ratio,User_Type
7157,A1P41TBZBOLTY1,0.70,10,0.48,0.33,49,0.20,"Explorer (Low Overlap %, Many other items)"
1219,AMP7TQRWAIE84,0.53,11,0.52,0.28,129,0.09,"Explorer (Low Overlap %, Many other items)"
4230,A3AL8GQ69QE7WN,0.71,6,0.29,0.20,39,0.15,"Explorer (Low Overlap %, Many other items)"
39,A14GK0E64J0WAS,0.79,5,0.24,0.19,34,0.15,"Explorer (Low Overlap %, Many other items)"
3578,A2IK776FY6MEMG,0.70,4,0.19,0.13,22,0.18,"Explorer (Low Overlap %, Many other items)"


# Case Study 2

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import os


WORK_DIR = '/kaggle/working/User_CF_Results_MeanCentered'
os.makedirs(WORK_DIR, exist_ok=True)


TARGET_USER_ID = 'AAE2DUEMTR30I' 

print(f"Case Study 2: Mean-Centered CF for {TARGET_USER_ID}")


if 'df_clean' not in globals() or 'sparse_matrix' not in globals():
    raise ValueError("Filtered data not found. Please run Iterative Filtering step first.")


if TARGET_USER_ID not in user_to_idx:
    TARGET_USER_ID = list(user_to_idx.keys())[0]
    print(f"Switched target to: {TARGET_USER_ID}")


print("\n[1/4] Calculating User Means and Centering Matrix...")


user_sums = sparse_matrix.sum(axis=1).A1 
user_counts = sparse_matrix.getnnz(axis=1)


user_means = np.zeros(len(user_ids))
valid_mask = user_counts > 0
user_means[valid_mask] = user_sums[valid_mask] / user_counts[valid_mask]


matrix_centered = sparse_matrix.copy().astype(float)


rows, cols = matrix_centered.nonzero()


matrix_centered.data -= np.take(user_means, rows)

print(f"Target User Mean Rating: {user_means[user_to_idx[TARGET_USER_ID]]:.4f}")


print("\n[2/4] Calculating Mean-Centered Similarity...")

target_idx = user_to_idx[TARGET_USER_ID]
target_vec_centered = matrix_centered[target_idx]


mc_sim_scores = cosine_similarity(target_vec_centered, matrix_centered).flatten()

mc_neighbors_df = pd.DataFrame({
    'User_ID': user_ids,
    'MC_Similarity': mc_sim_scores,
    'User_Mean': user_means
})


mc_neighbors_df = mc_neighbors_df[mc_neighbors_df['User_ID'] != TARGET_USER_ID]
mc_neighbors_df = mc_neighbors_df.sort_values(by='MC_Similarity', ascending=False)


top_50_mc = mc_neighbors_df.head(50)
top_50_mc.to_csv(os.path.join(WORK_DIR, f'{TARGET_USER_ID}_MC_Neighbors.csv'), index=False)

print("Top 5 Mean-Centered Neighbors:")
display(top_50_mc.head(5))


print("\n[3/4] Predicting Ratings...")



neighbor_indices = [user_to_idx[uid] for uid in top_50_mc['User_ID']]
neighbor_weights = top_50_mc['MC_Similarity'].values
neighbor_means = top_50_mc['User_Mean'].values


neighbor_matrix_centered = matrix_centered[neighbor_indices]


numerator = neighbor_matrix_centered.T.dot(neighbor_weights)


binary_matrix = neighbor_matrix_centered.copy()
binary_matrix.data[:] = 1 
denominator = binary_matrix.T.dot(np.abs(neighbor_weights)) 

predicted_deviation = np.zeros(sparse_matrix.shape[1])
mask = denominator > 0
predicted_deviation[mask] = numerator[mask] / denominator[mask]


target_mean = user_means[target_idx]
final_predictions = predicted_deviation + target_mean


rated_items_indices = sparse_matrix[target_idx].indices
final_predictions[rated_items_indices] = 0


top_item_indices = final_predictions.argsort()[::-1][:50]

recs_list = []
for idx in top_item_indices:
    score = final_predictions[idx]
    
    if score > 0: 
        recs_list.append({
            'Item_ID': idx_to_item[idx],
            'Predicted_Rating': round(score, 4),
            'Deviation_from_Mean': round(predicted_deviation[idx], 4)
        })

recs_df = pd.DataFrame(recs_list)
output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_MC_Predictions.csv')
recs_df.to_csv(output_file, index=False)

print(f"\n[4/4] Done. Results saved to: {WORK_DIR}")
display(recs_df.head(10))

Case Study 2: Mean-Centered CF for AAE2DUEMTR30I
Switched target to: A9Q28YTLYREO7

[1/4] Calculating User Means and Centering Matrix...
Target User Mean Rating: 4.1000

[2/4] Calculating Mean-Centered Similarity...
Top 5 Mean-Centered Neighbors:


,User_ID,MC_Similarity,User_Mean
7159,A1HCCW38EQQBTY,0.26,4.96
5722,A3GS5WAQVO4CFB,0.26,5.25
138,A2A788AUOVBOL2,0.25,5.45
5720,A17P4YJ8E81WNX,0.25,3.88
1131,A22WY238ZYNOS1,0.25,5.50



[3/4] Predicting Ratings...

[4/4] Done. Results saved to: /kaggle/working/User_CF_Results_MeanCentered


,Item_ID,Predicted_Rating,Deviation_from_Mean
0,B00123IAVK,9.18,5.08
1,B0011Z1D8Q,9.06,4.96
2,B000WKT6B2,8.95,4.85
3,B000VZYYWY,8.43,4.33
4,B00137T3ES,8.10,4.00
5,B000W176MM,7.96,3.86
6,B00136PSAM,7.60,3.50
7,B000YN32BM,7.48,3.38
8,B000W20FY2,7.40,3.30
9,B0013DA8R8,7.10,3.00


In [55]:
import pandas as pd
import math
import os

WORK_DIR = '/kaggle/working/User_CF_Results_MeanCentered'
TARGET_USER_ID = 'AAE2DUEMTR30I'

print(f"Identifying Top 20% Neighbors (Mean-Centered) for: {TARGET_USER_ID}")

if 'mc_neighbors_df' not in globals():

    try:
        file_path = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_MC_Neighbors.csv')
    
        print("Variable 'mc_neighbors_df' not found. Using calculated scores from memory...")
    except:
        raise ValueError("Error: Previous results missing. Please run Step 1 (Calculate Mean-Centered Sim) first.")

if 'mc_sim_scores' in globals() and 'user_ids' in globals():
    df_scores = pd.DataFrame({
        'User_ID': user_ids,
        'Adjusted_Score': mc_sim_scores
    })
else:
    raise ValueError("Similarity scores missing from memory. Please re-run Step 1.")

df_scores = df_scores[df_scores['User_ID'] != TARGET_USER_ID]


df_positive = df_scores[df_scores['Adjusted_Score'] > 0].copy()


df_positive = df_positive.sort_values(by='Adjusted_Score', ascending=False)


total_positive_neighbors = len(df_positive)


top_k_count = math.ceil(total_positive_neighbors * 0.20)

print(f"Total users with Positive Correlation (>0): {total_positive_neighbors}")
print(f"Top 20% Selection Count: {top_k_count}")

top_20_mc_df = df_positive.head(top_k_count)


output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_MC_Top20_Neighbors.csv')
top_20_mc_df.to_csv(output_file, index=False)

print(f"\nSaved Mean-Centered Top 20% to: {output_file}")

print("\nTop 10 Neighbors (Adjusted Similarity):")
display(top_20_mc_df.head(10

Identifying Top 20% Neighbors (Mean-Centered) for: AAE2DUEMTR30I
Total users with Positive Correlation (>0): 649
Top 20% Selection Count: 130

Saved Mean-Centered Top 20% to: /kaggle/working/User_CF_Results_MeanCentered/AAE2DUEMTR30I_MC_Top20_Neighbors.csv

Top 10 Neighbors (Adjusted Similarity):


,User_ID,Adjusted_Score
0,A9Q28YTLYREO7,1.00
7159,A1HCCW38EQQBTY,0.26
5722,A3GS5WAQVO4CFB,0.26
138,A2A788AUOVBOL2,0.25
5720,A17P4YJ8E81WNX,0.25
1131,A22WY238ZYNOS1,0.25
6496,A3J8SZL4IN1D46,0.25
6500,A1OKLAQJVFREOQ,0.25
5204,A3BY2QW7T0401N,0.25
3525,A32XSKO7TOI0BH,0.24


In [ ]:
import numpy as np
import pandas as pd
import os

WORK_DIR = '/kaggle/working/User_CF_Results_MeanCentered'
TARGET_USER_ID = 'AAE2DUEMTR30I' 

print(f"Predicting Ratings (Mean-Centered Approach) for: {TARGET_USER_ID}")
if 'top_20_mc_df' not in globals() or 'matrix_centered' not in globals() or 'user_means' not in globals():
    raise ValueError("Error: Missing data (Neighbors, Centered Matrix, or User Means). Please run previous steps.")

if TARGET_USER_ID not in user_to_idx:
    TARGET_USER_ID = list(user_to_idx.keys())[0]
    print(f"Warning: Switched to {TARGET_USER_ID}")

target_idx = user_to_idx[TARGET_USER_ID]
target_mean = user_means[target_idx]

print(f"Target User Average Rating: {target_mean:.4f}")
print(f"Using {len(top_20_mc_df)} neighbors for prediction...")


neighbor_ids = top_20_mc_df['User_ID'].values
neighbor_weights = top_20_mc_df['Adjusted_Score'].values
neighbor_indices = [user_to_idx[uid] for uid in neighbor_ids]


neighbor_matrix_centered = matrix_centered[neighbor_indices]



numerator = neighbor_matrix_centered.T.dot(neighbor_weights)


binary_matrix = neighbor_matrix_centered.copy()
binary_matrix.data[:] = 1 

denominator = binary_matrix.T.dot(np.abs(neighbor_weights))


pred_deviation = np.zeros(matrix_centered.shape[1])
mask = denominator > 0
pred_deviation[mask] = numerator[mask] / denominator[mask]


final_predicted_ratings = pred_deviation + target_mean


if 'sparse_matrix' in globals():
    items_rated_indices = sparse_matrix[target_idx].indices
    final_predicted_ratings[items_rated_indices] = 0 
    print("Warning: Original sparse_matrix not found, skipping 'already rated' filter.")


top_indices = final_predicted_ratings.argsort()[::-1][:50]

recs_list = []
for idx in top_indices:
    score = final_predicted_ratings[idx]
    
    if score > 0.001: 
        recs_list.append({
            'Target_User': TARGET_USER_ID,
            'Item_ID': idx_to_item[idx],
            'Predicted_Rating': round(score, 4),
            'Base_Mean': round(target_mean, 2),
            'Predicted_Deviation': round(pred_deviation[idx], 4)
        })

recs_df = pd.DataFrame(recs_list)


output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_MC_Predictions.csv')
recs_df.to_csv(output_file, index=False)

print(f"\nPrediction Complete. Saved to: {output_file}")

if not recs_df.empty:
    print("\nTop 10 Recommendations (Mean-Centered):")
    display(recs_df.head(10))
else:
    print("No recommendations generated (Check neighborhood

Predicting Ratings (Mean-Centered Approach) for: AAE2DUEMTR30I
Target User Average Rating: 4.1000
Using 130 neighbors for prediction...

Prediction Complete. Saved to: /kaggle/working/User_CF_Results_MeanCentered/A9Q28YTLYREO7_MC_Predictions.csv

Top 10 Recommendations (Mean-Centered):


,Target_User,Item_ID,Predicted_Rating,Base_Mean,Predicted_Deviation
0,A9Q28YTLYREO7,B0012CCO0E,13.50,4.10,9.40
1,A9Q28YTLYREO7,B0012FAUV6,9.93,4.10,5.83
2,A9Q28YTLYREO7,B000VWMTHE,9.93,4.10,5.83
3,A9Q28YTLYREO7,B00136LQC6,9.62,4.10,5.52
4,A9Q28YTLYREO7,B00123IAVK,9.18,4.10,5.08
5,A9Q28YTLYREO7,B0011Z1D8Q,9.06,4.10,4.96
6,A9Q28YTLYREO7,B00123KDR4,8.71,4.10,4.61
7,A9Q28YTLYREO7,B00137GCI8,8.71,4.10,4.61
8,A9Q28YTLYREO7,B00137SPH4,8.71,4.10,4.61
9,A9Q28YTLYREO7,B00137QMFG,8.71,4.10,4.61


In [ ]:
import numpy as np
import pandas as pd
import math
import os

WORK_DIR = '/kaggle/working/User_CF_Results_MeanCentered'
TARGET_USER_ID = 'AAE2DUEMTR30I' 

print(f"Calculating DF and DS (Mean-Centered) for: {TARGET_USER_ID}")

if 'top_20_mc_df' not in globals() or 'sparse_matrix' not in globals():
    raise ValueError("Error: Neighbors dataframe or Sparse Matrix not found. Please run previous steps.")

if TARGET_USER_ID not in user_to_idx:
    TARGET_USER_ID = list(user_to_idx.keys())[0]
    print(f"Switched to: {TARGET_USER_ID}")

target_idx = user_to_idx[TARGET_USER_ID]
target_vector = sparse_matrix[target_idx]

num_target_ratings = target_vector.getnnz()

beta = math.ceil(0.30 * num_target_ratings)
beta = max(1, beta)

print(f"Target User Rated Items: {num_target_ratings}")
print(f"Threshold (Beta) @ 30%: {beta}")
print("Counting common items...")

neighbor_ids = top_20_mc_df['User_ID'].values
neighbor_indices = [user_to_idx[uid] for uid in neighbor_ids]

neighbor_matrix = sparse_matrix[neighbor_indices]

target_binary = target_vector.copy()
target_binary.data[:] = 1

neighbor_binary = neighbor_matrix.copy()
neighbor_binary.data[:] = 1
common_counts = target_binary.dot(neighbor_binary.T).toarray().flatten()

ds_df = top_20_mc_df.copy()
ds_df['Num_Common_Items'] = common_counts

ds_df['Discount_Factor'] = ds_df['Num_Common_Items'].apply(lambda x: min(x / beta, 1.0))

ds_df['Discounted_Similarity'] = ds_df['Adjusted_Score'] * ds_df['Discount_Factor']


ds_df = ds_df.sort_values(by='Discounted_Similarity', ascending=False)


output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_MC_DS_Neighbors.csv')
ds_df.to_csv(output_file, index=False)

print(f"\nCalculation Complete. Saved to: {output_file}")

print("\nTop 10 Neighbors after Discounting (Mean-Centered):")
cols = ['User_ID', 'Num_Common_Items', 'Discount_Factor', 'Adjusted_Score', 'Discounted_Similarity']
display(ds_df[cols].head(10

Calculating DF and DS (Mean-Centered) for: AAE2DUEMTR30I
Switched to: A9Q28YTLYREO7
Target User Rated Items: 70
Threshold (Beta) @ 30%: 21
Counting common items...

Calculation Complete. Saved to: /kaggle/working/User_CF_Results_MeanCentered/A9Q28YTLYREO7_MC_DS_Neighbors.csv

Top 10 Neighbors after Discounting (Mean-Centered):


,User_ID,Num_Common_Items,Discount_Factor,Adjusted_Score,Discounted_Similarity
0,A9Q28YTLYREO7,70.00,1.00,1.00,1.00
7159,A1HCCW38EQQBTY,12.00,0.57,0.26,0.15
2183,A3HU0B9XUEVHIM,14.00,0.67,0.18,0.12
4206,A24N1BAS3CU27H,12.00,0.57,0.10,0.06
1516,A200C7YQJ45LRR,7.00,0.33,0.17,0.06
3257,A23FQRKY8W2MM4,7.00,0.33,0.15,0.05
5720,A17P4YJ8E81WNX,4.00,0.19,0.25,0.05
3254,A3PCTD8QM1BIXI,7.00,0.33,0.14,0.05
14,AD3C29MKDX7V2,8.00,0.38,0.12,0.04
3385,AE31M52VLKOG6,11.00,0.52,0.08,0.04


In [ ]:
import pandas as pd
import math
import os


WORK_DIR = '/kaggle/working/User_CF_Results_MeanCentered'
TARGET_USER_ID = 'AAE2DUEMTR30I' # (Or the active user)

print(f"Selecting Top 20% Neighbors based on DS (Mean-Centered) for: {TARGET_USER_ID}")


if 'ds_df' not in globals() or ds_df.empty:
    print("Error: Discounted Similarity results not found. Please run the previous DF/DS calculation step.")
else:
   
    valid_neighbors = ds_df[ds_df['Discounted_Similarity'] > 0].copy()
    
    
    valid_neighbors = valid_neighbors.sort_values(by='Discounted_Similarity', ascending=False)
    
    total_valid = len(valid_neighbors)
    
    if total_valid == 0:
        print("No neighbors with positive Discounted Similarity found.")
    else:
        
        top_k_count = math.ceil(total_valid * 0.20)
        
        print(f"Total Valid Neighbors (Positive DS): {total_valid}")
        print(f"Selecting Top 20%: {top_k_count} neighbors")
        
       
        mc_ds_top20_df = valid_neighbors.head(top_k_count)
        
       
        output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_MC_DS_Top20.csv')
        mc_ds_top20_df.to_csv(output_file, index=False)
        
        print(f"\nSaved Mean-Centered DS Top 20% to: {output_file}")
        
        
        print("\nFinal List of Top Neighbors (Ordered by DS):")
        cols = ['User_ID', 'Num_Common_Items', 'Discount_Factor', 'Adjusted_Score', 'Discounted_Similarity']
        display(mc_ds_top20_df[cols]

Selecting Top 20% Neighbors based on DS (Mean-Centered) for: AAE2DUEMTR30I
Total Valid Neighbors (Positive DS): 130
Selecting Top 20%: 26 neighbors

Saved Mean-Centered DS Top 20% to: /kaggle/working/User_CF_Results_MeanCentered/AAE2DUEMTR30I_MC_DS_Top20.csv

Final List of Top Neighbors (Ordered by DS):


,User_ID,Num_Common_Items,Discount_Factor,Adjusted_Score,Discounted_Similarity
0,A9Q28YTLYREO7,70.00,1.00,1.00,1.00
7159,A1HCCW38EQQBTY,12.00,0.57,0.26,0.15
2183,A3HU0B9XUEVHIM,14.00,0.67,0.18,0.12
4206,A24N1BAS3CU27H,12.00,0.57,0.10,0.06
1516,A200C7YQJ45LRR,7.00,0.33,0.17,0.06
3257,A23FQRKY8W2MM4,7.00,0.33,0.15,0.05
5720,A17P4YJ8E81WNX,4.00,0.19,0.25,0.05
3254,A3PCTD8QM1BIXI,7.00,0.33,0.14,0.05
14,AD3C29MKDX7V2,8.00,0.38,0.12,0.04
3385,AE31M52VLKOG6,11.00,0.52,0.08,0.04


In [ ]:

WORK_DIR = '/kaggle/working/User_CF_Results_MeanCentered'
TARGET_USER_ID = 'AAE2DUEMTR30I' 

print(f"Predicting Ratings using Discounted Mean-Centered Similarity for: {TARGET_USER_ID}")

if 'mc_ds_top20_df' not in globals() or 'matrix_centered' not in globals() or 'user_means' not in globals():
    raise ValueError("Error: Missing data. Please run Steps 1 and 5 first.")

if TARGET_USER_ID not in user_to_idx:
    TARGET_USER_ID = list(user_to_idx.keys())[0]
    print(f"Switched to: {TARGET_USER_ID}")

target_idx = user_to_idx[TARGET_USER_ID]
target_mean = user_means[target_idx]

print(f"Target User Mean Rating: {target_mean:.4f}")
print(f"Using {len(mc_ds_top20_df)} DS-weighted neighbors...")

neighbor_ids = mc_ds_top20_df['User_ID'].values
neighbor_weights = mc_ds_top20_df['Discounted_Similarity'].values

neighbor_indices = [user_to_idx[uid] for uid in neighbor_ids]

neighbor_matrix_centered = matrix_centered[neighbor_indices]


numerator = neighbor_matrix_centered.T.dot(neighbor_weights)

binary_matrix = neighbor_matrix_centered.copy()
binary_matrix.data[:] = 1
denominator = binary_matrix.T.dot(np.abs(neighbor_weights))


pred_deviation = np.zeros(matrix_centered.shape[1])
mask = denominator > 0
pred_deviation[mask] = numerator[mask] / denominator[mask]

final_predictions = pred_deviation + target_mean



if 'sparse_matrix' in globals():
    items_rated = sparse_matrix[target_idx].indices
    final_predictions[items_rated] = 0


top_indices = final_predictions.argsort()[::-1][:50]

recs_list = []
for idx in top_indices:
    score = final_predictions[idx]
    if score > 0.001: 
        recs_list.append({
            'Target_User': TARGET_USER_ID,
            'Item_ID': idx_to_item[idx],
            'Predicted_Rating': round(score, 4),
            'Base_Mean': round(target_mean, 2),
            'Predicted_Deviation': round(pred_deviation[idx], 4)
        })

recs_df = pd.DataFrame(recs_list)

output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_MC_DS_Predictions.csv')
recs_df.to_csv(output_file, index=False)

print(f"\nPrediction Complete. Results saved to: {output_file}")

if not recs_df.empty:
    print("\nTop 10 Recommendations (Mean-Centered + Discounted):")
    display(recs_df.head(10))
else:
    print("No recommendations found (Neighbors might not have sufficient overlap or ratings).")

Predicting Ratings using Discounted Mean-Centered Similarity for: AAE2DUEMTR30I
Switched to: A9Q28YTLYREO7
Target User Mean Rating: 4.1000
Using 26 DS-weighted neighbors...

Prediction Complete. Results saved to: /kaggle/working/User_CF_Results_MeanCentered/A9Q28YTLYREO7_MC_DS_Predictions.csv

Top 10 Recommendations (Mean-Centered + Discounted):


,Target_User,Item_ID,Predicted_Rating,Base_Mean,Predicted_Deviation
0,A9Q28YTLYREO7,B00123IAVK,9.18,4.10,5.08
1,A9Q28YTLYREO7,B000WKT6B2,8.95,4.10,4.85
2,A9Q28YTLYREO7,B0013DC9TS,8.71,4.10,4.61
3,A9Q28YTLYREO7,B00137SPH4,8.71,4.10,4.61
4,A9Q28YTLYREO7,B00137QMFG,8.71,4.10,4.61
5,A9Q28YTLYREO7,B00137GCI8,8.71,4.10,4.61
6,A9Q28YTLYREO7,B00123KDR4,8.71,4.10,4.61
7,A9Q28YTLYREO7,B00137GCGK,8.71,4.10,4.61
8,A9Q28YTLYREO7,B00136JL8W,8.71,4.10,4.61
9,A9Q28YTLYREO7,B000W20FY2,8.26,4.10,4.16


# Case Study 3

In [62]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import os

WORK_DIR = '/kaggle/working/User_CF_Results_Pearson'
os.makedirs(WORK_DIR, exist_ok=True)

TARGET_USER_ID = 'AAE2DUEMTR30I'

print(f"Case Study 3: Computing Pearson Correlation (PCC) for {TARGET_USER_ID}")
if 'sparse_matrix' not in globals() or 'user_to_idx' not in globals():
    raise ValueError("Error: Sparse Matrix or Mappings not found. Please run the Iterative Filtering step.")

if TARGET_USER_ID not in user_to_idx:
    TARGET_USER_ID = list(user_to_idx.keys())[0]
    print(f"Switched to: {TARGET_USER_ID}")

print("\n[1/3] Calculating User Means and Centering Data...")

user_sums = sparse_matrix.sum(axis=1).A1
user_counts = sparse_matrix.getnnz(axis=1)

user_means = np.zeros(len(user_ids))
valid_mask = user_counts > 0
user_means[valid_mask] = user_sums[valid_mask] / user_counts[valid_mask]

matrix_pearson = sparse_matrix.copy().astype(float)
rows, cols = matrix_pearson.nonzero()

matrix_pearson.data -= np.take(user_means, rows)

print(f"Matrix Centered. Target User Mean: {user_means[user_to_idx[TARGET_USER_ID]]:.4f}")

print("\n[2/3] Computing Pearson Coefficients...")

target_idx = user_to_idx[TARGET_USER_ID]
target_vec_pearson = matrix_pearson[target_idx]

pcc_scores = cosine_similarity(target_vec_pearson, matrix_pearson).flatten()

print("\n[3/3] Saving Results...")

pcc_df = pd.DataFrame({
    'User_ID': list(user_to_idx.keys()),
    'PCC_Score': pcc_scores
})


pcc_df = pcc_df[pcc_df['User_ID'] != TARGET_USER_ID]


pcc_df = pcc_df.sort_values(by='PCC_Score', ascending=False)


output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_PCC_Neighbors.csv')
pcc_df.to_csv(output_file, index=False)

print(f"Done. Saved to: {output_file}")

print("\nTop 10 Neighbors by Pearson Correlation:")
display(pcc_df.head(10))

print("\nBottom 10 Neighbors (Negative Correlation / Opposite Taste):")
display(pcc_df.tail(

Case Study 3: Computing Pearson Correlation (PCC) for AAE2DUEMTR30I
Switched to: A9Q28YTLYREO7

[1/3] Calculating User Means and Centering Data...
Matrix Centered. Target User Mean: 4.1000

[2/3] Computing Pearson Coefficients...

[3/3] Saving Results...
Done. Saved to: /kaggle/working/User_CF_Results_Pearson/A9Q28YTLYREO7_PCC_Neighbors.csv

Top 10 Neighbors by Pearson Correlation:


,User_ID,PCC_Score
7159,A1HCCW38EQQBTY,0.26
5722,A3GS5WAQVO4CFB,0.26
138,A2A788AUOVBOL2,0.25
5720,A17P4YJ8E81WNX,0.25
1131,A22WY238ZYNOS1,0.25
6496,A3J8SZL4IN1D46,0.25
6500,A1OKLAQJVFREOQ,0.25
5204,A3BY2QW7T0401N,0.25
3525,A32XSKO7TOI0BH,0.24
5723,A3D2VIUT2HWP0Z,0.24



Bottom 10 Neighbors (Negative Correlation / Opposite Taste):


,User_ID,PCC_Score
13,A20ZVK7L91JS7L,-0.10
30,A2IJ7JFHB94EKX,-0.13
42,A2LNEZD3YBO12O,-0.13
2193,A1O90V9J864L7N,-0.13
4651,A3V5GY711ZL680,-0.13
4605,A3I1L5E4LEZ9RV,-0.14
22,A2A6LKO4CC39S0,-0.16
3524,A16X18QH3TK8GV,-0.17
3591,A1XCQVVMDL4KT8,-0.22
38,A2OAKHL0E0A4YQ,-0.22


In [ ]:
import pandas as pd
import math
import os

# Setup
WORK_DIR = '/kaggle/working/User_CF_Results_Pearson'
TARGET_USER_ID = 'AAE2DUEMTR30I' 

print(f"2. Identifying Top 20% Neighbors (Pearson) for: {TARGET_USER_ID}")


if 'pcc_df' not in globals():
    try:
        # Attempt to load from file if variable is missing
        file_path = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_PCC_Neighbors.csv')
        pcc_df = pd.read_csv(file_path)
    except:
        raise ValueError("Error: PCC data not found. Please run the PCC calculation step first.")

positive_neighbors = positive_neighbors.sort_values(by='PCC_Score', ascending=False)

total_valid = len(positive_neighbors)

if total_valid == 0:
    print("No neighbors with positive Pearson correlation found.")
else:

    top_k_count = math.ceil(total_valid * 0.20)
    
    print(f"Total Neighbors with Positive Correlation: {total_valid}")
    print(f"Selecting Top 20%: {top_k_count} neighbors")
    

    top_20_pcc_df = positive_neighbors.head(top_k_count)
    
    output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_PCC_Top20_Neighbors.csv')
    top_20_pcc_df.to_csv(output_file, index=False)
    
    print(f"\nSaved Top 20% Pearson Neighbors to: {output_file}")
    
    
    print("\nTop 10 Closest Users (Pearson Correlation):")
    display(top_20_pcc_df.head(

2. Identifying Top 20% Neighbors (Pearson) for: AAE2DUEMTR30I
Total Neighbors with Positive Correlation: 648
Selecting Top 20%: 130 neighbors

Saved Top 20% Pearson Neighbors to: /kaggle/working/User_CF_Results_Pearson/AAE2DUEMTR30I_PCC_Top20_Neighbors.csv

Top 10 Closest Users (Pearson Correlation):


,User_ID,PCC_Score
7159,A1HCCW38EQQBTY,0.26
5722,A3GS5WAQVO4CFB,0.26
138,A2A788AUOVBOL2,0.25
5720,A17P4YJ8E81WNX,0.25
1131,A22WY238ZYNOS1,0.25
6496,A3J8SZL4IN1D46,0.25
6500,A1OKLAQJVFREOQ,0.25
5204,A3BY2QW7T0401N,0.25
3525,A32XSKO7TOI0BH,0.24
5723,A3D2VIUT2HWP0Z,0.24


In [64]:
import numpy as np
import pandas as pd
import os


WORK_DIR = '/kaggle/working/User_CF_Results_Pearson'
TARGET_USER_ID = 'AAE2DUEMTR30I'

print(f"3. Predicting Ratings (Pearson Method) for: {TARGET_USER_ID}")



if 'top_20_pcc_df' not in globals() or 'matrix_pearson' not in globals() or 'user_means' not in globals():
    raise ValueError("Error: Missing data (Neighbors, Pearson Matrix, or User Means). Please run previous steps.")


if TARGET_USER_ID not in user_to_idx:
    TARGET_USER_ID = list(user_to_idx.keys())[0]
    print(f"Switched to: {TARGET_USER_ID}")


target_idx = user_to_idx[TARGET_USER_ID]
target_mean = user_means[target_idx]

print(f"Target User Baseline Average: {target_mean:.4f}")
print(f"Using {len(top_20_pcc_df)} neighbors...")


neighbor_ids = top_20_pcc_df['User_ID'].values
neighbor_weights = top_20_pcc_df['PCC_Score'].values


neighbor_indices = [user_to_idx[uid] for uid in neighbor_ids]


neighbor_deviations_matrix = matrix_pearson[neighbor_indices]



numerator = neighbor_deviations_matrix.T.dot(neighbor_weights)


binary_matrix = neighbor_deviations_matrix.copy()

binary_matrix.data[:] = 1 

denominator = binary_matrix.T.dot(np.abs(neighbor_weights))


pred_deviation = np.zeros(matrix_pearson.shape[1])
mask = denominator > 0
pred_deviation[mask] = numerator[mask] / denominator[mask]



final_predictions = pred_deviation + target_mean


if 'sparse_matrix' in globals():
    items_rated = sparse_matrix[target_idx].indices
    final_predictions[items_rated] = 0


top_indices = final_predictions.argsort()[::-1][:50]

recs_list = []
for idx in top_indices:
    score = final_predictions[idx]
    
    if score > 0.001: 
        recs_list.append({
            'Target_User': TARGET_USER_ID,
            'Item_ID': idx_to_item[idx],
            'PCC_Rating': round(score, 4),
            'Baseline': round(target_mean, 2),
            'Predicted_Lift': round(pred_deviation[idx], 4)
        })

recs_df = pd.DataFrame(recs_list)


output_file = os.path.join(WORK_DIR, f'{TARGET_USER_ID}_PCC_Predictions.csv')
recs_df.to_csv(output_file, index=False)

print(f"\nPrediction Complete. Saved to: {output_file}")

if not recs_df.empty:
    print("\nTop 10 Recommendations (Pearson):")
    display(recs_df.head(10))
else:
    print("No recommendations generated (Neighbors might not have shared enough unrated items)."

3. Predicting Ratings (Pearson Method) for: AAE2DUEMTR30I
Switched to: A9Q28YTLYREO7
Target User Baseline Average: 4.1000
Using 130 neighbors...

Prediction Complete. Saved to: /kaggle/working/User_CF_Results_Pearson/A9Q28YTLYREO7_PCC_Predictions.csv

Top 10 Recommendations (Pearson):


,Target_User,Item_ID,PCC_Rating,Baseline,Predicted_Lift
0,A9Q28YTLYREO7,B0012CCO0E,13.50,4.10,9.40
1,A9Q28YTLYREO7,B0012FAUV6,9.93,4.10,5.83
2,A9Q28YTLYREO7,B000VWMTHE,9.93,4.10,5.83
3,A9Q28YTLYREO7,B00136LQC6,9.62,4.10,5.52
4,A9Q28YTLYREO7,B00123IAVK,9.18,4.10,5.08
5,A9Q28YTLYREO7,B0011Z1D8Q,9.06,4.10,4.96
6,A9Q28YTLYREO7,B00123KDR4,8.71,4.10,4.61
7,A9Q28YTLYREO7,B00137GCI8,8.71,4.10,4.61
8,A9Q28YTLYREO7,B00137SPH4,8.71,4.10,4.61
9,A9Q28YTLYREO7,B00137QMFG,8.71,4.10,4.61
